# 1.1 🚀 Initialization and Configuration

* In this section, necessary libraries are imported, and the Semantic Kernel is configured with Azure OpenAI credentials loaded from a configuration file.

In [ ]:
#!import config/Settings.cs

In [ ]:
// Import Semantic Kernel
#r "nuget: Microsoft.SemanticKernel, 1.7.1"

In [ ]:
using Microsoft.SemanticKernel;

var builder = Microsoft.SemanticKernel.Kernel.CreateBuilder();

In [ ]:
using Kernel = Microsoft.SemanticKernel.Kernel;

var (model1, azureEndpoint1, apiKey1, gpt35TurboServiceId) = Settings.LoadFromFile(model: "gpt-35-turbo-instruct");
builder.AddAzureOpenAIChatCompletion(model1, azureEndpoint1, apiKey1, serviceId: gpt35TurboServiceId);

var (model2, azureEndpoint2, apiKey2, gpt4ServiceId) = Settings.LoadFromFile(model: "gpt-4-32k");
builder.AddAzureOpenAIChatCompletion(model2, azureEndpoint2, apiKey2, serviceId: gpt4ServiceId);

var kernel = builder.Build();

# 1.2 🚀 Semantic Function for Short Intent Extraction
* This section focuses on setting up the Semantic Function for extracting a concise user intent using a specific prompt.

In [ ]:
using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.TemplateEngine;
using Microsoft.SemanticKernel.Connectors;
using Microsoft.SemanticKernel.Connectors.OpenAI;


var aiRequestSettings = new OpenAIPromptExecutionSettings 
{
    ExtensionData = new Dictionary<string, object> { { "api-version", "2023-03-15-preview" } },
    Temperature = 0.5,
    MaxTokens = 200,
    TopP = 0.2
};

In [ ]:
var input = "I want to find top-10 books about world history";

In [ ]:
string skPrompt = @"ChatBot: How can I help you?
User: {{$input}}

---------------------------------------------

Return data requested by user: ";

var arguments = new KernelArguments()
{
    { "input", input },
};

var getShortIntentFunction  = kernel.CreateFunctionFromPrompt(skPrompt, aiRequestSettings);

var intentResult = await kernel.InvokeAsync(getShortIntentFunction, arguments);

Console.WriteLine(intentResult);

# 1.3 🚀 Semantic Function for Conversational Chat

* This section defines the Semantic Function for engaging in a more extended conversation with the chatbot, initializing the context, and executing the function.

In [ ]:
skPrompt = @"
ChatBot can have a conversation with you about any topic.
It can give detailed answer or say 'I don't know' if it does not have an answer.

{{$history}}
User: {{$userInput}}
ChatBot:";

Chat with the chatbot

In [ ]:
using Microsoft.SemanticKernel.ChatCompletion;

var chatHistory = new ChatHistory("You are a librarian, expert about books");

In [ ]:
var message = chatHistory.Last();
Console.WriteLine($"{message.Role}: {message.Content}");

In [ ]:
chatHistory.AddUserMessage("Hi, I'm looking for book suggestions");

In [ ]:
message = chatHistory.Last();
Console.WriteLine($"{message.Role}: {message.Content}");

In [ ]:
IChatCompletionService chatService = kernel.GetRequiredService<IChatCompletionService>(gpt4ServiceId);

var reply = await chatService.GetChatMessageContentAsync(chatHistory);

Console.WriteLine(reply);


In [ ]:
message = chatHistory.Last();
Console.WriteLine($"{message.Role}: {message.Content}");

In [ ]:
chatHistory.AddSystemMessage(reply.ToString());

In [ ]:
message = chatHistory.Last();
Console.WriteLine($"{message.Role}: {message.Content}");

# 1.4 🚀 Interactive Chat Function

* In this final section, an interactive chat function is defined, and several interactions with the chatbot are simulated. The complete chat history and the final user input are then printed. *

In [ ]:
Func<string, Task> Chat = async (string input) => {
    // Save new message in the context variables
    chatHistory.AddUserMessage(input);

    var reply = await chatService.GetChatMessageContentAsync(chatHistory);

    chatHistory.AddSystemMessage(reply.ToString());
    
    Console.WriteLine(reply);
};

In [ ]:
await Chat("I would like a non-fiction book suggestion about Greece history. Please only list one book.");

In [ ]:
await Chat("that sounds interesting, what are some of the topics I will learn about?");

In [ ]:
await Chat("Which topic from the ones you listed do you think the most popular?");

In [ ]:
foreach (var message in chatHistory)
{
    Console.WriteLine($"{message.Role}: {message.Content}");
}

# 1.5 🚀 Switching between models

* In this final section, an interactive chat function is defined, and several interactions with the chatbot are simulated. The complete chat history and the final user input are then printed. *

Call HuggingFace model by name:

In [ ]:
#r "nuget: Microsoft.SemanticKernel.Connectors.HuggingFace, 1.6.1-preview"

In [ ]:
using Kernel = Microsoft.SemanticKernel.Kernel;

var (model, _, apiKey, _) = Settings.LoadFromFile(model: "google/flan-t5-xxl");

#pragma warning disable SKEXP0020
builder.AddHuggingFaceTextGeneration(model, apiKey: apiKey);


var kernel = builder.Build();

In [ ]:
var input = "I want to find top-10 books about world history";

In [ ]:
string skPrompt = @"ChatBot: How can I help you?
User: {{$input}}

---------------------------------------------

Return data requested by user: ";

var arguments = new KernelArguments()
{
    { "input", input },
};

var getShortIntentFunction  = kernel.CreateFunctionFromPrompt(skPrompt, aiRequestSettings);

var intentResult = await kernel.InvokeAsync(getShortIntentFunction, arguments);

Console.WriteLine(intentResult);